# English Topic Modeling — Harry Potter Fanfiction

In these lessons, we're learning about a text analysis method called *topic modeling*. This method will help us identify the main topics or discourses within a collection of texts a single text that has been separated into smaller text chunks.

---

## Dataset

### Harry Potter Fanfiction (English)

In this particular lesson, we're going to use [Little MALLET Wrapper](https://github.com/maria-antoniak/little-mallet-wrapper), a Python wrapper for [MALLET](http://mallet.cs.umass.edu/topics.php), to topic model a CSV file with fanfiction stories from the website [Archive of Our Own (AO3)](https://archiveofourown.org/).

___

<div class="admonition attention" name="html-admonition" style="background: lightyellow;padding: 10px">  
<p class="title">Attention</p>  
    
<p>If you're working in this Jupyter notebook on your own computer, you'll need to have both the Java Development Kit and MALLET pre-installed. For set up instructions, please see <a href="http://melaniewalsh.github.io/Intro-Cultural-Analytics/Text-Analysis/Topic-Modeling-Set-Up.html">the previous lesson<a/>.  </p>
    
If you're working in this Jupyter notebook in the cloud via Binder/JupyterHub, then the Java Development Kit and Mallet will already be installed. You're good to go!  
     
</div>  

## Set MALLET Path

Since Little MALLET Wrapper is a Python package built around MALLET, we first need to tell it where the bigger, Java-based MALLET lives.

We're going to make a variable called `path_to_mallet` and assign it the file path of our MALLET program. We need to point it, specifically, to the "mallet" file inside the "bin" folder inside the "mallet-2.0.8" folder. 

In [1]:
path_to_mallet = '../mallet/bin/mallet'

If MALLET is located in another directory, then set your `path_to_mallet` to that file path.

## Install Packages

In [2]:
#!pip install little_mallet_wrapper
#!pip install seaborn
#To install the most updated version of little_mallet_wrapper:
#!!pip install git+https://github.com/maria-antoniak/little-mallet-wrapper.git

## Import Packages

In [134]:
import little_mallet_wrapper
import seaborn
import pandas as pd
import random
from pathlib import Path
pd.options.display.max_colwidth = 100

## Get Training Data From CSV File

In [239]:
hp_df_english = pd.read_csv("harry_potter_english_stories_first_chaps.csv")

In [240]:
hp_df_english

,work_id,title,author,rating,category,fandom,relationship,character,additional tags,language,...,status date,words,chapters,comments,kudos,bookmarks,hits,all_kudos,all_bookmarks,body
0,10057010,All the Young Dudes,['MsKingBean89'],Mature,M/M,Harry Potter - J. K. Rowling,"Sirius Black/Remus Lupin, Sirius Black & Remus Lupin, James Potter/Lily Evans Potter","Remus Lupin, Sirius Black, James Potter, Lily Evans Potter, Peter Pettigrew, Severus Snape, Mine...","Marauders' Era, Marauders, Marauders Friendship, wolfstar, Get Together, Slow Burn, so slow, it'...",English,...,2018-11-12,"526,969",188/188,"30,425","154,030","30,777","10,553,092","['Infired_DabDab', 'Ambick', 'molly_may46', 'CigarettesAndMoonysJumpers', 'cheersmoony', 'river2...",[],"Saturday 7th August, 1971\n\nHe woke up in the dark. It was too hot in the little room they'd pu..."
1,234222,Then Comes a Mist and a Weeping Rain,['Faith Wood (faithwood)'],Explicit,M/M,Harry Potter - J. K. Rowling,Draco Malfoy/Harry Potter,"Harry Potter, Draco Malfoy, Gregory Goyle, Millicent Bulstrode, Ernie Macmillan, Hermione Grange...","Humor, Romance, Hogwarts Eighth Year, HP: EWE, Post - Deathly Hallows, Complete",English,...,2011-08-04,"21,139",1/1,990,"58,627","11,301","700,250","['orth0sie', 'NonScaryGirl', 'anyexcuse', 'dmwildflower', 'lilgreengremlin', 'tsukk1sdinofr1es',...",[],"\n\nHis cloak was soaked within minutes. Three Impervius Charms, several Drying Spells and one ..."
2,14454174,Manacled,['senlinyu'],Explicit,F/M,Harry Potter - J. K. Rowling,Hermione Granger/Draco Malfoy,"Hermione Granger, Draco Malfoy, Dolores Umbridge, Tom Riddle | Voldemort, Astoria Greengrass, Gr...","Post-War, Harry Potter Dies, Alternate Universe - Voldemort Wins, Forced Pregnancy, Imprisonment...",English,...,2019-08-19,"370,515",77/77,"11,956","57,777","16,878","3,942,936","['sweetcherrydestiel', 'Jjthebug', 'Claudialoner', 'lonelyjuicebox', 'Big_wing101', 'KawaiiFairi...",[],\n (See the end of the chapter for notes.)\n \n\n\n\n\n\nHermione had long gi...
3,46452973,The Sword Saint and Satella,['Reactioonist'],Mature,F/M,"Re:zerokaraShi meruYi Shi Jie Sheng Huo | Re:Zero Starting Life in Another World (Anime), Highs...","Natsuki Subaru/Rem, Emilia/Natsuki Subaru, Beatrice & Natsuki Subaru","Emilia (Re:Zero), Natsuki Subaru, Rem (Re:Zero), Ram (Re:Zero), Frederica Baumann, Garfiel Tinse...",NaN,English,...,2023-04-13,"31,682",3/?,18,"49,072",NaN,"3,824","['Rynlikesstories', 'Titticacca', 'whorn0524', 'killerman1215', 'Echidna_the_witch_cannot_be_thi...",[],'How is it even possible?' A body lying on the ground thought. His head and body lying to the si...
4,3101099,The Standard You Walk Past,['bafflinghaze'],Mature,M/M,Harry Potter - J. K. Rowling,Draco Malfoy/Harry Potter,"Draco Malfoy, Harry Potter, Hermione Granger, Ron Weasley, Narcissa Black Malfoy, Pansy Parkinso...","Hogwarts Eighth Year, Some Homophobia in the Wizarding World, Romance, Friends to Lovers, Fluff,...",English,...,2015-03-12,"46,202",14/14,"3,370","47,942","9,257","774,057","['hisokook', 'abbieq11', 'Anya_Trevelyan', 'sapphicgingers', 'Wisteria14', 'am_yvny', 'Hypnotisi...",[],\n (See the end of the chapter for notes.)\n \n\nDraco recast the ward over h...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
780,4065484,A Good Teacher,['lecturience'],General Audiences,Gen,Harry Potter - J. K. Rowling,NaN,"Harry Potter, Dudley Dursley, Petunia Evans Dursley","Time Travel, Mentor/Protege, Pre-Hogwarts",English,...,2015-06-03,"12,989",1/1,367,"6,540","1,663","49,079","['Aquira_Jobel', 'JxeTee', 'ReadsALot59', 'clumsy_and_robotic', 'Tresken02', 'NotaPridefulWitch'...",[],Harry had been looking forward to school for the first time ever. Not only did it mean he got a...
781,1300453,Son Of The Champion,['sar_kaz_m'],Mature,"M/M, F/M, Gen","Marvel Cinematic Universe, The Avengers (Marvel Movies), Harry Potter - J. K. Rowling",Clint Barton/Phil Coulson,"Phil Coulson, Clint Barton, Harry Potter, Nick 

In [155]:
hp_df_english['body'] = hp_df_english['body'].astype(str)

### Pre-Process Texts

`little_mallet_wrapper.process_string(text, numbers='remove')`

Next we're going to process our texts with the function `little_mallet_wrapper.process_string()`. This function will take every individual post, transform all the text to lowercase as well as remove stopwords, punctuation, and numbers, and then add the processed text to our master list `training_data`.

In [156]:
training_data = []
for text in hp_df_english['body']:
    processed_text = little_mallet_wrapper.process_string(text,                                                        numbers='remove                                                      #stop_words_extra=spanish_stop_words)
    training_data.append(processed_text)

In [157]:
original_texts = [text for text in hp_df_english['body']]

In [158]:
hp_english_titles = [title for title in hp_df_english['title']]

In [178]:
hp_english_relationship = [relationship for relationship in hp_df_english['relationship']]

In [179]:
hp_english_characters = [chars for chars in hp_df_english['character']]

In [180]:
hp_english_tags = [tags for tags in hp_df_english['additional tags']]

### Get Dataset Statistics

We can get training data summary statisitcs by using the funciton `little_mallet_wrapper.print_dataset_stats()`.

In [159]:
little_mallet_wrapper.print_dataset_stats(training_data)

Number of Documents: 785
Mean Number of Words per Document: 3810.0
Vocabulary Size: 43054


## Training the Topic Model

We need to make a variable `num_topics` and assign it the number of topics we want returned. Then we're going to set a file path where we want all our MALLET topic modeling data to be dumped.

In [160]:
# your choice of topics
num_topics = 50

path_to_mallet = '../mallet/bin/mallet'

#No need to change anything below here
training_data = training_data

#Set output directory
output_directory_path = 'topic-model-output/hp-fanfiction/english-firstchaps'

#Create output directory
Path(f"{output_directory_path}").mkdir(parents=True, exist_ok=True)

#Create output files
path_to_training_data           = f"{output_directory_path}/training.txt"
path_to_formatted_training_data = f"{output_directory_path}/mallet.training"
path_to_model                   = f"{output_directory_path}/mallet.model.{str(num_topics)}"
path_to_topic_keys              = f"{output_directory_path}/mallet.topic_keys.{str(num_topics)}"
path_to_topic_distributions     = f"{output_directory_path}/mallet.topic_distributions.{str(num_topics)}"

### Train Topic Model

Then we're going to train our topic model with `little_mallet_wrapper.quick_train_topic_model()`.

In [161]:
#little_mallet_wrapper.quick_train_topic_model(path_to_mallet,
#                                            output_directory_path,
#                                             num_topics,
#                                             training_data)

Importing data...
Complete
Training topic model...
Complete


([['voldemort',
   'death',
   'dark',
   'dumbledore',
   'lord',
   'would',
   'boy',
   'dead',
   'life',
   'kill',
   'one',
   'soul',
   'die',
   'eyes',
   'world',
   'body',
   'master',
   'killed',
   'blood',
   'like'],
  ['though',
   'little',
   'time',
   'janice',
   'sighs',
   'feeling',
   'lyona',
   'cup',
   'think',
   'coffee',
   'fingers',
   'fact',
   'cold',
   'onto',
   'stanley',
   'kitchen',
   'strange',
   'watches',
   'last',
   'coat'],
  ['draco',
   'weasley',
   'granger',
   'malfoy',
   'father',
   'mother',
   'look',
   'pansy',
   'manor',
   'okay',
   'ridiculous',
   'gryffindor',
   'give',
   'room',
   'every',
   'mark',
   'closer',
   'smiling',
   'shut',
   'death'],
  ['harry',
   'potter',
   'back',
   'still',
   'another',
   'together',
   'watching',
   'hand',
   'best',
   'matter',
   'wand',
   'instead',
   'glasses',
   'slightly',
   'towards',
   'place',
   'okay',
   'able',
   'hard',
   'saw'],
  ['newt

When the topic model finishes, it will output your results to your `output_directory_path`.

## Display Topics and Top Words

To examine the topics that the topic model extracted from the training data, run the cell below. This code uses the `little_mallet_wrapper.load_topic_keys()` function to read and process the MALLET topic model output.

In [162]:
topics = little_mallet_wrapper.load_topic_keys(path_to_topic_keys)

for topic_number, topic in enumerate(topics):
    
    print(f"✨Topic {topic_number}✨\n\n{topic}\n")

✨Topic 0✨

['voldemort', 'death', 'dark', 'dumbledore', 'lord', 'would', 'boy', 'dead', 'life', 'kill', 'one', 'soul', 'die', 'eyes', 'world', 'body', 'master', 'killed', 'blood', 'like']

✨Topic 1✨

['though', 'little', 'time', 'janice', 'sighs', 'feeling', 'lyona', 'cup', 'think', 'coffee', 'fingers', 'fact', 'cold', 'onto', 'stanley', 'kitchen', 'strange', 'watches', 'last', 'coat']

✨Topic 2✨

['draco', 'weasley', 'granger', 'malfoy', 'father', 'mother', 'look', 'pansy', 'manor', 'okay', 'ridiculous', 'gryffindor', 'give', 'room', 'every', 'mark', 'closer', 'smiling', 'shut', 'death']

✨Topic 3✨

['harry', 'potter', 'back', 'still', 'another', 'together', 'watching', 'hand', 'best', 'matter', 'wand', 'instead', 'glasses', 'slightly', 'towards', 'place', 'okay', 'able', 'hard', 'saw']

✨Topic 4✨

['newt', 'graves', 'percival', 'tina', 'omega', 'theseus', 'alpha', 'pickett', 'grindelwald', 'man', 'omegas', 'scamander', 'creatures', 'alphas', 'niffler', 'sir', 'credence', 'gonzales', 

## Load Topic Distributions

MALLET also calculates the likely mixture of these topics for every single document in the corpus. This mixture is really a probability distribution, that is, the probability that each topic exists in the document. We can use these probability distributions to examine which of the above topics are strongly associated with which specific documents.

To get the topic distributions, we're going to use the `little_mallet_wrapper.load_topic_distributions()` function, which will read and process the MALLET topic model output.

In [186]:
topic_distributions = little_mallet_wrapper.load_topic_distributions(path_to_topic_distributions)

In [187]:
topic_distributions[0]

[0.006367726766720324,
 2.2770696756764044e-05,
 0.0021204449221878534,
 0.0015835783478129325,
 1.6970103462882967e-05,
 3.9796917933367056e-05,
 0.004041163116671068,
 1.6991865743782836e-05,
 0.008860887256736755,
 6.730942960256957e-05,
 0.0783530690223081,
 0.00016116990313225265,
 0.020208938585088,
 0.030645089478169588,
 0.03824789816904002,
 8.966857388295575e-05,
 0.0020718987289805677,
 3.195984487498598e-05,
 1.8525676439279054e-05,
 0.006151083809602592,
 0.00014781323352047493,
 0.0023405181830522376,
 6.053195807499654e-05,
 0.1364109123255422,
 0.06608711009051423,
 0.09590033983393198,
 0.05292394779072493,
 7.542347372446853e-05,
 0.03550372395524137,
 0.0200071499718794,
 0.0003430263737682546,
 0.023394514798954196,
 0.00010699015311706742,
 8.493692526766681e-05,
 0.038585697117505015,
 0.0051183658388672315,
 0.0011759919515335006,
 0.0030249568061611848,
 0.00022680228505318496,
 0.04856378400352355,
 0.006651062682925397,
 0.021790712079476544,
 0.04514982183872

## Display Top Documents Per Topic

In [232]:
from IPython.display import Markdown, display
import re

def make_md(string):
    """A function that transforms string data into Markdown
    so it can be nicely formatted with bolding and emojis
    """
    display(Markdown(str(string)))

def get_top_docs(docs, topic_distributions, topic_index=1, n=5, doc_length = 2000):
    
    """A function that shows the top documents for a given set of topic distributions
    and a specific topic number
    """
    
    sorted_data = sorted([(_distribution[topic_index], _document) for _distribution, _document in zip(topic_distributions, docs)], reverse=True)
    topic_words = topics[topic_index]
    make_md(f"### ✨Topic {topic_index}✨\n\n{topic_words}\n\n---")
    
    for probability, doc in sorted_data[:n]:
        # Make topic words bolded
        for word in topic_words:
            if word in doc.lower():
                doc = re.sub(f"\\b{word}\\b", f"**{word}**", doc, re.IGNORECASE)
        make_md(f'✨  \n**Topic Probability**: {probability}  \n**Document**: {doc[:doc_length]}\n\n')

# Work, Coffee, Tea

In [233]:
get_top_docs(original_texts, topic_distributions, topic_index=30, n=10)

### ✨Topic 30✨

['time', 'table', 'work', 'tea', 'coffee', 'home', 'two', 'day', 'drink', 'week', 'floo', 'another', 'together', 'three', 'get', 'months', 'across', 'always', 'half', 'years']

---

✨  
**Topic Probability**: 0.1320793426267969  
**Document**: TRIGGER WARNING for Chapter 1: Dark thoughts/brief mention of suicide and suicidal thoughts


            (See the end of the chapter for  more notes.)
            






January 2007


Tangled sheets and sweat, followed by sleeplessness and shivering. It would be dramatic stuff indeed if it weren't so commonplace these days. Draco Malfoy finally stopped trying to find a comfortable sleeping position and settled onto his back. Gray eyes opened and stared at the ceiling, counting handsome panels of an impeccably furnished bedroom. It was probably dawn, or close to it. He rolled over to grab his wristwatch off the mahogany side **table**. 4:46 in the morning. How nice of his nightmares to let him have a bit of a lie-in.

You could call him afraid of sleep if you wanted at this point, but he was in no hurry to try and slumber just now. He had just awoken from one of his absolute favorite trips down memory lane: the one where the Muggle Studies professor gets eaten by a giant snake on his family's dining room **table**, mere feet from him.

Draco threw his legs over the edge of his bed, and put his head in his hands between his knees. Deep breath. And **another**. And **another**. You're still breathing. You're still here. I am in control of this.

Dressed in a black suit for **work** and seated at the end of a long table, he found the energy to raise a cup of **tea** to his lips every few minutes. The impressive display of breakfast foods lay untouched before him. The house elves had, as they did every morning, clearly tried to entice him, but it was no use. Draco knew it would feel like ash in his mouth, and he wouldn't keep it down anyway.

He existed trance-like, lingering over a long-cold saucer of **tea** until past 7:30. Coffee. He could muster the energy for a good, strong cup of **coffee**. Keep your routine. I am in control of this.

The smell of freshly brewed hot **coffee** was one of the scents Draco recognized wafting out of the cauldron of Amortentia in P



✨  
**Topic Probability**: 0.12118609483814656  
**Document**: Harry sat hunched over his **work** in the middle of an overcrowded, overpriced **coffee** shop. He was rushing to finish an essay that was due to his professor later that afternoon. Harry **always** told himself that he would stop procrastinating, but he would then procrastinate stopping. It was a complicated affair.

He was about to **get** up when suddenly a man who looked to be about his age came crashing into his **table**, looked around for a moment, then asked "Do you have a boyfriend?"

Harry looked up at him briefly, completely bewildered, before replying "No."

"Wonderful. You do now." The other man promptly sat down, took a book out of his bag and began reading.

Harry was still dumbstruck but had finally closed his jaw and began to appreciate the features on the blond-haired, grey-eyed person now sitting so close to him. He was about to ask him his name, considering he didn't know it, when **another** unknown individual casually approached.

"Draco, dear, finally managed to start that book club I never wanted to join with you?" He was a tall and slim dark skinned man, probably a couple of **years** older than the **two** who were sitting.

"Actually Blaise," the one sitting with Harry, Draco apparently, put his book down and took a sip of Harry's **coffee**, "this is my boyfriend, who doesn't mind that I like to read." He had put his hand over Harry's and Harry could feel himself blush.

"Oh come on now, you act as though we merely broke up because all you wanted to do was talk about some dumb author."

Draco looked a bit pained but rolled his eyes to brush off the insult, "You should be going now, we have plans to head back to my place and, well, NOT talk about books." He smiled sweetly and drew his hand back when Blaise exited the shop.

"Sorry about that," Draco began to stand, "I saw him trailing me and the only way to **get** him off my back would be to intimidate him with someone attractive."

"So you think I'm attractive?" Harry had allowed the we



✨  
**Topic Probability**: 0.11982786976449153  
**Document**: "Do you do **coffee**?"

Remus looks up from his battered copy of Kings Canyon and blinks. Currently he's perched behind the counter of Lily's Roundtable Cafe, sub-heading: 'Finest Coffee in Fairford'. Not only do they do **coffee**, they do the best damned coffee in the whole of the Cotswolds. At least, that's what Lily so oft proclaims.

Apparently, though, someone has missed that.

"We do," says Remus.

He sets down his feet and stands, flashing the customer an automatic smile. It melts into something more genuine when he looks properly at the person standing behind the battered oak counter he and Lily salvaged from a removals shop in Swindon **three** **years** ago.

The customer is tall and slender, with a lightly tanned face and fiercely sharp features Remus doesn't have to look twice at to be taken aback by. His jet black hair - pulled back into a loose ponytail - and ever-so-slightly strange accent make him suddenly seem very exotic. It's an image heightened by the bunch of long necklaces dangling **across** his t-shirt, among which are dog tags, a prayer cross and a dulled shark's tooth.

He has on a black leather biker jacket too. After tearing his eyes from the man for only a fleeting moment, Remus spots the hulking great motorcycle parked outside. So engrossed he'd been in Kings Canyon, he hadn't even heard the beast of a machine roll up.

The man is very good-looking. He is also definitely not from the Cotswolds.

"Great, I'm gagging," he smiles, exposing a brilliant flash of white teeth. "What's it like?"

"What's...?"

"The coffee."

"Oh, it's - it's the Fairford coffee in finest," Remus burbles. "I mean! The finest, the finest coffee in Fairford."

"That sounds alright," the man replies, eyes sparkling with what might be amusement. "I'll have that."

"Brilliant." Clumsily, Remus reaches to grab a clean mug. "Anything else?"

The man shakes his head no, flashing Remus **another** quick smile before ambling over to one of the six empty tables in the ca



✨  
**Topic Probability**: 0.11949491086789238  
**Document**: 
          (See the end of the chapter for  notes.)
          

Harry smiled, reached out and tucked a stray strand of longish dark-blond hair behind Stefan's ear. Stefan instinctively lowered his head, and Harry clicked his tongue to show his disapproval, though it was more humourous than a true feeling of annoyance.

"You can look at me, pet," he encouraged, and Stefan slowly lifted his head to reveal a hesitant but beautifully innocent smile. Harry reached for Stefan's hand and circled his thumb gently over the pulse point on his wrist, caressing it to offer support and a sense of calm in Stefan.

"Caleb is a good man, he'll take good care of you, Stefan. You do know that, don't you? We've discussed it at length, but please do let me know if there's anything that's making you uncomfortable. Remember, you are in control here," he reminded the young, slightly insecure man who stood before him.

Stefan nodded in silent understanding.

"It's all okay, Harry; it really is, I'm just nervous, that's all."

Harry smiled. He appreciated the honesty. Having spent the last **three** **months** training Stefan, teaching him all he needed to know about being submissive, and especially being a submissive for Caleb, he cared deeply for his charge. Compared to his previous arrangements, it hadn't been all that much of a challenge since Stefan was naturally submissive, relished in following orders and just needed a little guidance, as well as a steady hand. Mind, he was a bit of a brat, but it wasn't anything Harry couldn't handle.

They had both met with Caleb a few times over the last **two** weeks, and both Caleb and Stefan seemed rather smitten with each other, although Stefan was still somewhat shy whenever he was around his new Dom; it was rather endearing to watch him steal the one or other glance whenever he thought Caleb's attention was directed elsewhere. Harry had no doubt that both Caleb and Stefan wouldn't take long to **get** properly acquainted. He had known Caleb 



...

✨  
**Topic Probability**: 0.1042274223840116  
**Document**: Remus had figured out a plan for the morning ahead of him: check in at the uni book store to see if one of the course books he'd been waiting on had arrived yet, make a quick stop off at the **coffee** shop to pass his banking form to Lily and discuss a tentative schedule, and **get** back to campus in **time** for his second class of the **day**. He hadn't, however, been planning to leave after the second item on his list with a heavy head or his heart beating several registers faster than usual, but all in all, Remus never made plans for that as a general rule. 

One could call the lengthy queue at the bookstore, at first, the only real hiccup to his plan. He'd been expecting a queue, but not one so long it seemed as though every student at school had been between classes when he turned up. It was a little stuffy in there too, but he read while he waited and managed to **get** out of there with what he'd come for before it got to be a bit too much. He caught the bus heading away from campus, feeling grateful for the **time** he'd allotted for travel, and walked **across** the parking lot toward the shop with ten minutes to spare. 

Once inside, he stood in the doorway briefly debating his next course of action. He wasn't entirely sure whether he should inform one of the baristas of his arrival or expected to head straight to the backroom, but it didn't take long for him to decide that the latter would be a bit too presumptuous and wandered over to wait behind a couple standing ahead of the till.  

Remus generally tended to run quite cold and often wore multiple layers to accommodate the fact, but he had not been prepared for the temperature to peak as high as it had over the course of the morning. The air-conditioning running through the cafe was an improvement to the book store certainly, but he did wish he'd gone and found a spot to set his things down while he waited. He toyed with the idea of removing some of his layers, but his book bag and laptop carrier we



✨  
**Topic Probability**: 0.09820488030455231  
**Document**: 
          (See the end of the chapter for  notes.)
          

James Potter just likes to think that he's down on his luck. It happens to everyone at some point, right? Down on your luck sounds better than directionless, on the verge of overdrawing your bank account, jobless, 23-year-old. That's what he tells himself as he shuffles quickly into the little **coffee** shop on the corner, rubbing his hands **together** rapidly to warm them up. The little bell chimes, announcing his entrance, and his glasses immediately fog up due to the rapid change in temperature. 

The scent of **coffee** brewing is a welcomed one, and James closes his eyes, inhaling deeply and letting the roasted warm smell comfort him in its familiarity. Gentle music is playing somewhere from the hidden speakers in the shop.  

"Look, I hate to be this person because I love seeing you in here every **day** I do," the barista began wryly from the other side of the counter, "but I'm afraid Melting Cup Cafe and Bakery is bleeding you dry money-wise."

"Remus," James opened his eyes and quickly cleared the fog on his glasses with his shirt before putting them back on. He greeted the barista with his best charming smile that never seemed to falter, no matter how down on his luck he was. "You have my usual?" 

Remus sighed turning around before placing a large cup on the counter next to the register. Steam was coming off the little hole in the lid and James felt his caffeine headache melt away just looking at it. "That's PS4.09," Remus replied flatly, ringing him up. 

James began fumbling in his pockets looking for his wallet, checking his jacket pockets first then the pockets in his trousers. "Ah," he says when he finally finds it, pulling it out and paying. 

"I almost feel that, ethically, I cannot keep selling you coffee when you have no money and no job," Remus shook his head as James took his first sip.

"I am working on the job part," James grinned. "One of these days I'm going to be a millionai



✨  
**Topic Probability**: 0.09817224502740789  
**Document**: Harry nursed the beer in front of him as the sounds of the busy bar rolled over him. He'd lost count of the amount of alcohol that he'd consumed as he sat there. The burn in his stomach and his slow thinking told him that he'd had enough, **two** drinks back. He just lifted up the bottle and sipped at the beer.

He didn't react any more than glance up at the man that sat down at the **table** **across** from him and placed **two** beers down. His facial hair was well-groomed under his nose and chin but had a wild look to the rest of his hair. His eyes glinted with interest and he was wearing a top of the line dress shirt, jacket and pants. He gave off the sense of a playboy dressed up like a businessman on the town. Harry barely gave him a glance as the man pushed one of the beers towards him with an arrogant smile.

"Something I can help you with?" Harry asked, sarcastically.

"Possibly," the man replied, with an American accent. "My name is Tony Stark."

Harry just waved his hand as he drank the rest of his beer in one go, ignoring the one that Tony had placed down.

"Nice to know," Harry stated as he stood. "Table is all yours."

The man paused at that as Harry gave him a nod and weaved his way out of the bar. Once someone found him, he knew his **time** of being anonymous was over. Harry walked out onto the street, followed by the smell of smoke and the thudding beat of music. His steps weren't steady but he made do as he staggered away from the bar.

"Hey!" Tony called out and Harry glanced back. "I brought you a **drink**."

"Great for you," Harry yelled back and kept walking.

"At least give me your name," Tony tried.

Harry turned back to the man, unsure why he was being so insistent. Now that he knew that Tony wasn't aware of who he was. Tony stepped forward smoothly and Harry took that in as well as the arrogant smile on the man's lips. Harry gave him a once over and that smile widened. He knew what the man was after.

"I can see that you're interested," T



✨  
**Topic Probability**: 0.09690904063289764  
**Document**: Hermione had almost made a clean escape until her unfortunate discovery on the street. Confronted with the reality that her coat pockets were, indeed, empty--neither the crumpled receipt from the grocer's nor a slightly sticky sweet wrapper could be mistaken for her mittens--she was forced to return to the Ministry, entirely against her will.

She needed those mittens because she'd promised Ginny and Angelina that she'd take the kids ice skating tomorrow, giving their parents a much-needed break. Fred, Roxanne, and James were prone to rampaging for hours, and a Warming Charm on her bare hands wasn't going to cut it in a Muggle park. No, she needed her mittens, the ones that she'd hand-knitted **two** winters ago. The stitching wasn't the best, and she'd had to redo one of the thumbs **three** times; but they were warm, comfortable, and imbued with an exceptionally strong Warming Charm. She hadn't even been out of the Ministry for ten minutes, and the bitter night air had already numbed her bare fingers.

Hermione hurried through London's snowy streets, squeezing herself into the musty old telephone box and silently cursing the Ministry's Anti-Disapparition Jinx the entire ride down. Oh, there were excellent reasons for the jinx's existance, but, at the moment, what she wanted was to pop in and out of her office in a heartbeat. She'd originally hoped to escape from the Ministry before anyone in particular had noticed that she was gone, as he'd been making little remarks about tonight's Christmas party. Well, she wasn't going to be around for this year's party, that was for sure. Not after what had happened last **time**.

As the telephone box came to a halt, Hermione looked up and down the entrance hall before cautiously leaving the booth. Although the entrance hall usually held no more than a few people by seven o'clock at night, tonight it was completely deserted thanks to those either tucking into their dinner at **home** or into the Ministry's gratis champagne at



✨  
**Topic Probability**: 0.09682934126418064  
**Document**: 
November, 1998


The common room was quiet, finally. Draco sat down in an armchair in front of the fire with a satisfied sigh. Grabbing a **table**, he spread out his Ancient Runes books and parchment, and vaguely wondered if he should summon a house elf to ask for some **tea**. He shook his head; if someone were to come down, he'd need to be able to pack up his things and move on quickly, and **tea** would only slow that process down. Salazar, how he hated this new shared common room. At least if he was down in the dungeons he could be ignored without having to give up his customary seat by the fire. But up here in the west tower, he had to contend with stray Ravenclaws and Gryffindors and even the odd Hufflepuff for Salazar's sake. The other Slytherins just ignored Draco these days, but everyone else seemed to enjoy trying to **get** a rise out of him.

McGonagall was obviously out of her Scottish mind, shoving all the returning Eighth Years into a House all of their own and expecting everyone to just **get** along. The last couple of **months** had been torture for Draco and the other Slytherins who had decided to try and finish their education. There was only the four of them: Pansy, Blaise, Draco, and the Greengrass girl none of the other **three** had ever talked to much. Out of the four, Draco got it the worst, for obvious reasons, although nobody seemed inclined to let Pansy's mistakes go either. Although to be fair, she had been rather vocal about them. Pansy had taken to also ignoring Draco, probably in the hopes that it would endear her to the rest of their classmates. Draco thought she was wasting her **time**, but he didn't make it harder for her. He understood where she was coming from; his own mistakes had been much larger.

Draco sighed tiredly and opened his textbook. Once the prince of Slytherin House, now relegated to waiting for the common room to clear out before being able to sit down and finish his homework on his own. Really, the only thing 



# Family

In [229]:
get_top_docs(original_texts, topic_distributions, topic_index=13, n=5)

### ✨Topic 13✨

['child', 'mother', 'boy', 'father', 'children', 'parents', 'son', 'baby', 'little', 'lily', 'family', 'name', 'ollie', 'james', 'old', 'magic', 'world', 'house', 'mum', 'young']

---

✨  
**Topic Probability**: 0.24130958323896284  
**Document**: 
          (See the end of the chapter for  notes.)
          

James Potter couldn't breathe.

Looking up at him, from the arms of the nurse that had just been by his wife's side, was an infant with big brown eyes. 


 A **baby** **boy**. 



 His **baby** **boy**. 


Sirius and Remus were there instantly to steady him and he looked at the nurse as if she was able to work miracles. "This-"

"Congratulations, Mr. Potter, you're a **father**... Have you picked out a **name**?"

It itched his brain that the nurse hadn't asked him when Lily was with him, as she'd had as much a hand in picking names... but he was too overwhelmed to start that discussion with the nurse, instead just smiling dopily. "Charles. Charles James Potter." 


 It was only by a bare inch (and respect to his **parents**) that Lily had accepted the tradition of naming their first-born after the parent of their gender. 


"Lily, she-... she's okay?" Regaining his composure slightly, James reached out for the **child**, momentarily losing his train of thought at having this **world**-changing being in his arms. "... can I see my wife now?"

He could feel the smiles on Sirius and Remus' faces, knowing he'd never live down having to be removed from Lily's room during labor because of the panic he'd been in, but it didn't even matter right now. He just wanted to see his wife and share this moment with her. 

"Mr. Potter..." 

It was the tone of voice more than anything that had him looking up with something like apprehension building in his gut. 

The nurse smiled, but it was strained. "Your second **child** is having complications and it would be in everyone's best interest for you to stay out here."

"My sec-" His brain whirled. 


 Twins. He had twins. Lily was still in that white room while he was waiting out here, unknowing of anything that she was going through and- 


He felt Remus take the child from his arms and then he proceeded to faint.

 

Helena Ivy Potter was born almost 12 hours after her



✨  
**Topic Probability**: 0.23596362661000123  
**Document**:  


THE HEIR OF THE HOUSE OF BLACK


 

 

 

 

Welcome newcomers and hello familiar faces,


For newcomers, I hope you enjoy my first-ever written creative work. This will be a long story and I hope you can join me on this journey! I love comments, so please let me know your thoughts!

 

For those who are revisiting this story or rereading due to the author's note published on February 5, 2022 (Chapter 62), this story has undergone a major revision over the past two months. If you started reading this story before February 5, 2022, then please note that you might have read the **old** version. So, please proceed with caution or consider restarting the story. If you started reading after February 5, 2022, then rest assured that you have read the updated/ revised version. Finally, thank you for your continued support for my story.

 

Best regards,

ravenclawblues 2022-02-05

 

 CHAPTER ONE


PROLOGUE: TURAIS ORION BLACK


 

         The cold air hit the newborn **baby** uncomfortably as he cringed and let out an involuntary cry. His eyes were shut tightly as his small ears picked up the first sounds of this **world** - voices chatting, fabrics shifting, feet shuffling.

         "A healthy screamer, this one," the wet nurse said. She bathed and cleaned the **baby** before wrapping it in soft linen. Carefully, she handed the **child** off to the impatient **father**, who immediately soaked in and marvelled at every feature of his newborn **child** with the softest gaze.

         "Is it a **boy**?" the **mother** asked immediately. However, the **father** was too preoccupied with caressing the child's cheek to pay any attention. Instead, the wet nurse replied with the affirmation. "Thank the **magic** that it is a **boy**."

         In the father's warm embrace, the baby slowly cracked open his bleary eyes a mere fraction of an inch.

         Peering up, the first thing he saw of the **world** was a handsome man. The corners of his father's eyes were crinkled w



✨  
**Topic Probability**: 0.2231133650890256  
**Document**: 
Petunia Evans had lived her entire life so far, or at least since age eight or so, with one goal: to be as normal as possible. To blend in with the most neutral of crowds until she would never have to worry about standing out, and the way she walked, the way she talked, the way she existed, would all come naturally as perfectly average.



This may have seemed like a strange choice for such a **young** girl to make, but a view inside the Evans household would make everything perfectly clear. Mr. and Mrs. Evans had been perfectly normal for years themselves, a businessman and a school teacher with two lovely daughters and a lovely, normal home. Their daughters attended a nice school, came home to nice meals, had nice friends from nice families, and everything went normally.



The two girls would rise for school int he morning, Petunia typically waking her sister Lily as well so that the younger girl wouldn't sleep right through first hour, and they would take the school bus at exactly seven-forty-five and get to school at eight o'clock for the eight-fifteen bell. They went through their day, eating lunch at the same table, and rode the same bus home at two-fifteen, where they would watch the telly until Mrs. Evans returned home at three-thirty and made them do their coursework.



Petunia was pleased as punch.



Then everything changed.



Strange occurrences began to surround the Evans **family**, and they could all be traced back to Lily. Their teacher's hair turned blue when Lily was upset with her, the toys would float around the **house**, and the strange **boy** down the street taught her how to make flowers blossom and reverse in her palm. Just like that, their **family** would never be normal again.



Mr. and Mrs. Evans, despite their pleasant, normal, respectable life, harboured a deep love for all things interesting and quirky and exciting, and so they latched onto the idea of **magic** or magical powers. To think that their lovely daughter might be ble



✨  
**Topic Probability**: 0.22040092462124544  
**Document**: This first chapter is set six years before the start of All the Young Dudes, so there are no real spoilers, and it can be read at any time. Might pack a bit more of a punch if you're up to date with ATYD though.


          (See the end of the chapter for  notes.)
          



While I'm far away from you, my **baby**





I know it's hard for you, my **baby**.





Because it's hard for me my baby





And the darkest hour is just before dawn.



 


"You've never been a very clever girl, Hope."



"No, **mum**." Hope stared into her cup of tea. No milk, just a sunny slice of lemon, served in a proper teacup with a saucer that had a matching roseleaf design. She'd been supposed to receive a similar set when she'd got married, but Lyall hadn't wanted muggles at the reception.



Her **mother** tutted loudly.



"I always said he was no good. Man like that - no **family**, no church. And you never explained exactly what he did for work."



"He was in local government." Hope replied. She put the teacup down on the **little** end table in her **mother**'s living room. 



"Council?" Her **mum** asked, brightening a bit, "Well that's something. Did he leave a pension? Anything at all?"



"A **little** bit. But I want to keep it by for Remus."



Her mother tutted again. She thought it was a silly **name**. Hope had tried to compromise, and given her **son** her own **father**'s **name** too - but Remus John sounded even worse, according to her mother.



Mrs Jenkins preferred to pretend that Hope's little **boy** didn't exist at all, even when he was sleeping in the bedroom upstairs. Hope wanted to go and check on him now - give him a cuddle - but she didn't dare get up; her mother would call it coddling and Hope didn't want a fight. He was sleeping a lot - that was probably normal for five year olds. 



But Remus wasn't a normal five year **old**, not any more.



A pain struck Hope deep inside her chest; heartbreak. She bowed her head, letting her hair fall forward



✨  
**Topic Probability**: 0.17844542177358502  
**Document**: 

A/N: Hello, people!





I don't own Harry Potter.





I have no beta.





ENJOY!





CHECK ME OUT ON TUMBLR. HELLY-WATERMELONSMELLINFELLON.




~.O.~


"Having magical twins is such a blessing!" a brown-haired Medi-Witch sighed dreamily as she eyed the two babies sleeping in Lily's arms.

"They always share an ability or talent. The Weasley twins are apparently telepathically linked and very talented at wandless **magic**," one of the Healers told them.

"The Carrow twins are rumoured to be Metamorphs and can Astral Project themselves."

"The Patil twins are both natural Occulmens, and can conjure items easily."

Lily and James stared down at their daughters. Not identical exactly. Basically, they looked the same, save for their hair colour. Belladonna had darker hair than even James, and Prunella took after Lily more. And if the usual 'blessings' occurred, then both would some day share two magical abilities because that was how **magic** worked. 'Twice-Blessed' was the term used for such cases.

Belladonna Pandora Potter and Prunella Alula Potter were going to be extra special. They were sure of it.

"I just wish this wasn't so gross," said Lily, wrinkling her nose at the sight of the bottles that were filled with the proper potions, as well as their blood in order to make Sirius' blood adoption legal. The babies each had to drink an entire bottle in order for the ritual to work.

Sirius shrugged. "This is for their safety in the long run. Some things are worth it."

The thought of drinking blood at all made her stomach churn, but she said no more.

Belladonna took to her bottle easily, but Prunella wailed and had to be coaxed into drinking her own. James and Lily both cooed and petted her hair until it was all gone.

The Goblins said nothing.

The harsh screaming of one of their daughters alerted them to trouble. Both of the new **parents** rushed into the nursery where they found Prunella holding her head where a large red bump was swelling above her brow.



# Goblins

In [234]:
get_top_docs(original_texts, topic_distributions, topic_index=9, n=5, doc_length = 4000)

### ✨Topic 9✨

['goblin', 'goblins', 'magical', 'gringotts', 'bank', 'ollivander', 'vault', 'family', 'wizard', 'black', 'alley', 'dragon', 'magic', 'dumbledore', 'blood', 'school', 'diagon', 'griphook', 'wizards', 'vaults']

---

✨  
**Topic Probability**: 0.1936727837809281  
**Document**:  The World Was Young

"What is this?"

The boy looks up. He's been wandering by himself for a long time. His stomach aches, and he thinks that his aunt is going to be so angry with him. But he can't find the way back by himself, he really can't.

Bending over him is what looks like an ugly person. But the boy is used to his uncle and cousin being ugly, and he just looks at the person's jagged teeth and narrowed eyes and claws on his hands. He doesn't flinch when the person reaches out and picks him up. He thinks that maybe they're going to take him home.

"Why are you here?" the person asks him.

People rarely ask the boy questions. He has to think about it before he finally answers, "I went out of the garden."

"You're lost?"

The boy nods. That ought to explain it, he thinks, and now the person will take him home.

But the person just goes on staring at him, as if lost boys are strange. Then he reaches up and brushes away the boy's hair. The boy flinches. He's sure, he's sure, that any second the person is going to laugh at his scar. His aunt is always telling him how ugly it is. Sometimes the boy thinks he would like it if she didn't call it ugly, but that never happens.

"Your name is Harry Potter?"

The boy hesitates. "I don't know?"

"You must know. Human children are old enough by your age to know their names!"

"Well, I mean, I just get called boy and freak," the boy explains, even though he doesn't like explaining it. "Maybe my name is Harry Potter? I think Aunt Petunia called me Harry once."

The person closes his eyes and mutters to himself in what sounds like another language. The boy listens with interest. "Can I learn that language?" he asks, when the person falls silent and opens his eyes again.

"Listen to me. I called you a human child. Don't you know what that means?"

"You're not human?"

The person starts to answer, and then pauses. "And that doesn't bother you?"

The boy shakes his head. He feels a little hopeful. Maybe he's found a freak, like him. Maybe the person will take him away to the land where all the freaks live, and then he'll be with people like him and won't have to wash the dishes or weed the garden again. "Can I come with you to where your land is?"

The person just keeps staring at him. Then he asks, "What are your relatives like?"

"They don't like me. They call me freak. I sleep in a cupboard--"

"What?"

"A cupboard," the boy repeats, glaring at the person. He can't tell the truth if the person keeps interrupting! "It's under the stairs. My cousin has two bedrooms, but they say I can't have one. Because I'm a freak."

The person chatters his teeth together sharply. It makes a noise like something dangerous. The boy hopes he can learn to do that, too. Before he can ask the question, the person smiles. "So they won't come looking for you?"

"I don't think so. They don't know where I am right now."

The person nods and says, "Then you can come with me. There is a land for my kind--we are called **goblins**--and I think we should have the charge of raising you. Since your guardians have done such a poor one." He laughs deep down in his chest. The boy thinks the sound is kind of scary, but it makes the **goblin**'s chest underneath him vibrate pleasantly.

"Okay. Can I learn your language? Can I chatter my teeth like you do?"

The **goblin** nods and begins to carry him away. The people walking by don't seem to notice them, the boy realizes. It's as if they have a **magical** bubble traveling with them. "You may need some help with the teeth. But you will receive all the help you need, and more, from my clan."

*

"I suppose you don't know what you're doing."

"Of course I do."

The goblin who rescued the boy is called Ripclaw, he has learned. He thinks it's a wonderful name. He plays next to the desk in the office that Ripclaw has brought him to. They don't think he's listening, but he is. The boy learned how to listen when he was young. Sometimes that means he can avoid Uncle Vernon or Aunt



✨  
**Topic Probability**: 0.18506068042355658  
**Document**: Harry finds out he has been lied to after going in for cleansing and tries to claim his Heir/Lordship rings.


          (See the end of the chapter for  notes.)
          

Harry was utterly furious and devastated as he looked down at the results of the Inheritance Test he had just taken. Having arrived at Diagon Alley just last night, after he had run away from home after blown up his Aunt Marge, Harry had decided to head to Gringotts to take the test. Having overheard Seamus telling Dean that by taking the test it could reveal if the Muggleborn boy's absent father was in fact Magical as well as line he might belong to. Harry decided to do it in hope of finding out more about his **family** than what most people tended to tell him, which usually was that he looked like his Dad but had his Mother's eyes, or that his dad was good at Quidditch.

 

He had gone in to take the test his second day in the **alley** which seemed to make the Goblins happy as they told him they had been trying to reach him for the last two years but each of their letters to have been returned unopened. Harry had been mystified for he had not known of any mail that he had not gotten from the Magical World after his Uncle had tried to outrun his Hogwarts' letter. Harry was lead into a back room where the test took place before the Goblin left him alone with his results to set up a cleansing for him.

 


Harry James Potter


Parents:

Godparents:


Family Lines:



Enchantments/Potions on the person in order of age:



Vault discrepancies:


 

Harry could not believe that his two best friends have been stealing from him since before they even became friends, let alone Mrs. Weasley and Professor Dumbledore who had shown him more love and kindness than any adult he had ever meet. Also, he could not figure out why there was a spell on him to change his eyes or hair color and what it could accomplish. Harry of course knew about the Glamour spell being on him having applied it on himself before he had gone to his First-Year at Hogwarts to hide the signs of abuse his **family** had done to him. He was glad that the Goblins had offered to give him a cleansing just based on the length of his results as he was to get the cleansing. As he was getting ready to go through the cleansing a Goblin informed him after he woke back up he would also be given his Heir/Lordship rings which should have been done during his first visit giving him another reason to distrust Hagrid more than just having the large man being on his results of someone he was enchanted to trust.

 

Seething with anger at all the betrayal that had been done to him Harry fell unconscious to go through the cleansing. He was still angry when he woke up in a warm bed and the smells he associated with the Hospital Wing. Other than the anger he was still filled with Harry felt lighter and more powerful than he ever felt before. Swinging his legs over the bed he was on he saw that someone had set up a mirror beside his head. Climbing to his feet he walked over to the mirror and gasp at seeing his normally **black** rat nest hair now a deeper red than any of the Weasleys and now hung smoothly down to his shoulders. The only other change he saw was that his eyes were no longer vivid green but bright blue it took him a few moments as he took in his new look before he realizes that he was not actually wearing his glasses. Unfortunately, the lightning bolt scar was still there, as were several scars given to him by his Uncle peaking out over the top of his shirt. He wished he could apply his Glamor but did not want to cast underage **magic** and get in trouble again.

 

"Ah Heir Potter you are awake I hope you feeling better now," a Goblin in pale green robes said peeking their head into the room. "If you follow me Account Manager Fangtooth is ready to give you your Heir/Lordship rings." Harry followed what he assumed to be a Healer Goblin to an ornately carved door and pushed who pushed it aside to reveal a



✨  
**Topic Probability**: 0.17498524001706042  
**Document**: Shouta wanted to be mad.

He'd expected to be mad, in all honesty. Shouta had been simmering in slowly increasing levels of anger for at least a week.

He'd started by being irritated a week ago when a testy and exhausted owl had interrupted his class to drop an official looking roll of parchment on Shouta's lap before turning to Koda and the birdseed the boy kept in his desk. He'd then been mad two days ago when he was sitting on a plane on his way to England and away from his responsibilities. He'd progressed to being angry four hours ago while walking across the most obnoxious lobby he'd ever seen to meet people with the fakest smiles he'd ever seen (and Shouta worked in U.A. while occasionally having to meet with the Hero Commission).

Anger had been a foregone conclusion from the beginning; if his **family** didn't want his squib ass when he was born, not to mention his mother's and grandfather's, then they shouldn't want him now.

But Shouta had also been curious. Just a bit. Because they hadn't wanted anything to do with him before, and his limited interaction with Japan's **magical** world said that shouldn't have changed. Hizashi had always liked to say that it was Shouta's curiosity that led him to underground heroics, as much as his intolerance for idiocy and desire to help those that so often got missed.

Regardless, he'd entered the **bank** angry. Three hours after sitting through this ridiculous meeting with the English Ministry of Magic, Gringotts Bank, three teenagers, and sundry other adults whom apparently had no purpose, and Shouta felt safe in saying he was no longer mad. Which was a shame, because mad would be much easier to handle.

No, Shouta was downright pissed. 

These Ministry **wizards** had brought him across the damn globe to try and use his mother's bloodline to gain access to the Black **family** accounts for reparations. Apparently, Shouta was a direct descendent even if from a squib line, while Potter, one of the teenagers, was named heir only through the Will of an escaped convict. The same convict who'd also removed the only other people with Black **blood** from the family tree in a way that was so permanent that it actually topped what had been done to Shouta's squib grandfather, Marius.

Shouta didn't like being dragged across the ocean (away from his Hell class, Hizashi said they were doing fine, which was impossible; the man must be lying). Shouta didn't like the thought of paying for reparations for a ministry and **bank** he wasn't a part of, all for a fight he's also played no part in finishing. The significant amount they assured him would be left over for personal use didn't really change his mind (and it wasn't suspicious at all how they knew exactly how much was in a private account).

He also didn't particularly like the part where the vast majority of people in this room, particularly Madam Hopkirk and her superior Pius Thicknesse, were clearly planning on the squib being too dumb to ask the right questions.

He loathed, however, with a coiling fury, the part where two teenagers and one thin red-headed man in worn robes were the only people who seemed remotely on the legal heir's side. And that included the kid himself (17 was still a fucking kid no matter what their backwards laws said). Potter looked like he wanted to say nothing more than fuck the money, throw it at Shouta, and go hide under a particularly think blanket for the next month or six.

Shouta was pretty confident in his assessment of the kid, since Potter's eyes kept drifting over to Shouta no matter who was speaking at the time. And, kami, those eyes. They were a brilliant green that shone only more starkly against the purple and white paleness of his skin.

Shouta knew exactly how tired you had to be to get bags like that.

They also shone with a look that Shouta had only seen as Eraserhead. Only seen when someone was so sure no help was coming that right up until the end they couldn't truly buy into the idea 



✨  
**Topic Probability**: 0.1707876916098019  
**Document**: 
After the Battle of Hogwarts, Harry Potter helped with the rebuilding and then went back to **school**, sitting his NEWTS and graduating with the rest of what 

The Daily Prophet

 called 'The Disrupted Class.' Following that, everyone expected him to become an auror. Harry himself expected it, right up until the moment came to apply to the academy.



Harry found himself standing and staring at the Ministry of Magic's rebuilt atrium. The statue of **wizards** and witches dominating the other races that had been put there by Voldemort's followers was long gone. Instead, there was a statue of Harry.



Harry hadn't known they'd done that.



He was standing tall, shoulders squared, the Sword of Gryffindor held high over his head in a dramatic pose. A heroic pose. His cloak flared out around him, his hair fluttering majestically in the same imaginary wind. On his face was a look of grim determination combined with an unshakable conviction.



Harry didn't think it looked a thing like him. In fact, he was certain that he'd never looked like that, and never would. And, with a sinking feeling, he realized that this was how the wizarding world saw him, and he could either start trying to live up to it right now, and accept the backlash when he inevitably faltered, or he could dig his heels in and adamantly refuse.



Harry was already in the floo back to Grimmauld Place before he registered his decision.



-l-



In the next three years, Harry became something of a recluse. He was never seen in public, and could only rarely be coaxed into a Weasley **family** dinner. He didn't like to go to those too often, as Ginny seemed to still be waiting for him, and he didn't think it was fair to pursue a relationship with her when he barely ventured into the wizarding world.



Of course, his hermitism seemed to have just increased the public's fascination with him. Rumors abounded that he was traveling and learning rare and wonderful **magic**, or else had become an Unspeakable, or else was hunting down rogue Death Eaters, or gone to find the fabled High Elves and marry their princess or something. The harder he tried to make himself forgettable, the more **wizards** everywhere were determined to regard him as the return of Merlin.



There was some truth in the rumors. He was learning rare and wonderful **magic**, if only for something to do. He'd worked his way steadily through Grimmauld Place's library, and whenever he found a subject that caught his fancy, he'd send Kreacher 'round to the other properties he'd inherited to find more books on the subject. Ron thought he'd gone absolutely mental, while Hermione despaired that he'd lacked this drive while they were in **school**. And then she huffed when he refused to let her help with whatever his latest project was, and would start chattering about her work at the Ministry.



Harry'd found that when he wasn't in a 'learn or die' situation, that he much preferred moving at his own pace, and got a certain feeling of satisfaction in having sorted something out for himself.



His project at the moment was learning the animagus transformation, which Hermione would have kittens about if she knew, as all his references advised against attempting such a thing without another witch or **wizard** supervising. But Harry figured he was safe enough with Kreacher keeping an eye on him, as they'd grown quite fond of each other, and he wanted to do this for himself. In a way, this was his tribute to the Marauders. To his father and Sirius. He'd show Ron and Hermione once he'd managed it and had enjoyed his form for a bit.



He was hoping to be something inconspicuous so that he could go out without the possibility of people recognizing him.



"Alright, Kreacher, you know what to do," Harry said, as he sat in a room he'd cleared of all furnishings. He wanted to give himself plenty of space, in case he was something large. He doubted he'd be too big for the room, unless he was an elephant. But really



✨  
**Topic Probability**: 0.17048976326773488  
**Document**: 
          (See the end of the chapter for  notes.)
          

As Harry took in the contents of his **vaults** he was shocked about how much money he had after all his Aunt always complained how much money he cost him to take care of not that she tended to use much since he was forced to wear Dudley's old clothes, and barely got any food. This made him think of the times when he would clean his Uncle's office and had caught a glimpse of the **family**'s checkbook when his Aunt left it out. He had noticed that his Uncle actually earned a lot of money from his job at Grunnings yet they were still getting some money once a month from an unknown source. So as Hagrid recovered from the next ride on the mine cart to the **vault** to pick up his package from the **school** Harry moved over to their guide. Not knowing what was the proper title he decided to just guess, "Master Goblin," only to stop at the startled look on the Goblin's face. "I am sorry I do not know your proper title," Harry said with a slight bow.

"It is alright young Wizard it is just rare to be shown such respect from your kind," Griphook said giving him what Harry guessed was a kind smile.

"Um, I was just wondering if it would be possible to get information on the status of my **vault**?" Harry asked.

"That can be arranged all you need to do is make a return visit to....," Griphok said but stopped when he saw the Wizard's face fall "will that be a problem?"

"Well I live with my **family** who do not have **magic** and I don't think I will be able to get back anytime soon," Harry explained. For though he was mostly left alone at home apart from his chores there would be no way that he could get back to London before he was to go to class and he somehow knew he should see them before he went to Hogwarts.

"That is actually easy to deal with before you leave I will give you a **magical** artifact known as a Portkey which will allow you to transfer back to the **bank** when you have some free time. I must say it is rather strange that your family did not come with you for even a Muggleborn, that is a Magical child from a non-**magical** family, usually is brought to the **bank** by both a teacher as well as their members of their family," Griphook said.

"Well my Aunt is not a fan of Magic and my Uncle is never home anyway," Harry said.

"Alright, we will make sure to give you one that is both easy to hide as well as easy to pass as an everyday object," Griphook said making a note on a small pad that he pulled out of his vest. By that time Hagrid had recovered enough from the ride to collect the contents of the vault for the **school** to end their conversation as Harry had learned long ago not to judge a book by its covers. He thought this applied to Hagrid twice over for the man was so large he appeared to be rather intimidating yet acted so kind and carrying but it could all be an act. After all his Aunt and Cousin would act as the perfect family to his uncle but if anything went wrong even if it was Dudley breaking something it was blamed on him. His Uncle whose job keep him out of the house most of the time would believe them over Harry who he was told was a violent troublemaker. When in truth Harry would rather be left alone luckily he found such solitude at the local library only staying at home to do his morning chores then going home at night to make supper, which due to some "problem he had caused" during the day he was never allowed to actually eat. His Uncle, however, never wanted to not feed him so he would at the very least get some bread and water unless his Aunt lied and said he had eaten before Vernon got home leading him to go hungry until breakfast. After Hagrid collected the package the small group of three rode back to the top level of the bank and as Hagrid once more took a little while to settle down from the ride Griphook took out an envelope which contained the Portkey as well as for instructions on how to use it and handed it to Harry.





In [230]:
get_top_docs(hp_english_titles, topic_distributions, topic_index=9, n=5)

### ✨Topic 9✨

['goblin', 'goblins', 'magical', 'gringotts', 'bank', 'ollivander', 'vault', 'family', 'wizard', 'black', 'alley', 'dragon', 'magic', 'dumbledore', 'blood', 'school', 'diagon', 'griphook', 'wizards', 'vaults']

---

✨  
**Topic Probability**: 0.1936727837809281  
**Document**: Music Beneath the Mountains



✨  
**Topic Probability**: 0.18506068042355658  
**Document**: Harry Lord of Slytherin



✨  
**Topic Probability**: 0.17498524001706042  
**Document**: Heroes all have a Saving People Thing



✨  
**Topic Probability**: 0.1707876916098019  
**Document**: The Dragon of Moria



✨  
**Topic Probability**: 0.17048976326773488  
**Document**: One Quiet Question



# Sex

In [235]:
get_top_docs(original_texts, topic_distributions, topic_index=41, n=5, doc_length = 4000)

### ✨Topic 41✨

['cock', 'mouth', 'hand', 'fingers', 'hips', 'around', 'arse', 'neck', 'hard', 'want', 'kiss', 'tongue', 'hands', 'body', 'fuck', 'hair', 'one', 'skin', 'lips', 'inside']

---

✨  
**Topic Probability**: 0.23383806387394368  
**Document**: There's a patch of something dark and viscous on the back wall, dried and flaking off - he hopes it's not blood. The men sitting at the table below it don't even seem to notice it. One of them is driving the blade of a pen knife into the table, repeatedly stabbing it into the worn wood, his gaze fixed a foot to the left of the blade, his eyes glassy. His companion holds his pint right below his **mouth**, blowing gently over the **mouth** of the bottle, his right **hand** down the front of his jeans, his eyes fixed on Draco.

Draco's **skin** crawls. He slowly raises the tumbler of whiskey to his mouth, willing himself not to examine the water marks on the glass again. The surface of the bar is sticky and the barkeep leaning his hip into the bar with his face buried in a pornographic magazine doesn't seem to be inclined to wipe it. There's a dark, heavy beat thumping from the jukebox in the corner; a young man and woman, clearly riding something more potent than just alcohol, sway together, completely out-of-synch with the music, the guy's **hand** firmly cupped over the woman's bottom, her skirt riding up to reveal skimpy knickers in a shocking pink.

Draco can feel the pervert's eyes on him like cigarette butts being put out on his **skin**. Shivering lightly, he drains his drink in **one** gulp, swallowing and quickly clearing his throat against the burn of the truly horrible quality of the alcohol, while simultaneously earning the barkeep's attention.

The sweaty man sighs, grabs a bottle off the shelf and shuffles over, the magazine with the naked woman on the cover still held firmly in his other hand. Her nipples are dark and round, her breasts abnormally large - it makes Draco a little queasy.

The man pours Draco's drink, puts the bottle down with a thump, and opens the magazine where he stands; there's a bulge in his trousers and he shifts from foot to foot as he leans back against the counter behind the bar, adjusting his boner.

Draco shivers again, resisting the impulse to just go home, scrub off the top layer of his skin under a scalding hot shower, and go to bed. He needs this - he needs tonight; tonight has got to be good.

The door opens as a patron enters and there's a soft tinkle. Draco restlessly looks **around**.

His breath hitches.

Harry fucking Potter.

The Saviour stands there, dark blue jeans, fashionably faded, a black t-shirt that clings to his muscular, tapering torso, and a well-worn black leather jacket over it, pushing his square, black framed spectacles up his nose, his **hair** tousled in a way that makes Draco think of heated writhing on rumpled sheets, hot breath, flushed skin and **fingers** desperately carded through **hair**.

Okay, so Potter now apparently just happens to go **around** looking like every wet dream Draco has ever had.

Potter shoves his **hands** into his pockets, looking around, and then saunters forward, slowly walking towards the bar. He pauses at a stool several seats away from Draco, and carefully looks down the bar, his eyes landing squarely on Draco sitting there, his eyes determinedly fixed on his drink.

Draco shuts his eyes and drags in two deep breaths before he's assaulted by Potter's scent -something warm and spicy and so determinedly male, it instantly sends a frisson of something utterly shameless right down to Draco's **cock**.

He presses his knees together, his hand trembling as he clenches it around his glass.

"Are you lost?" Potter murmurs, and Draco actually starts because he hadn't realised just how close the man is standing. His warm breath washes over Draco's ear, tickling his **neck** and sending more signals to his **cock** to wake up and stand at full mast.

"Potter," he says with as much dignity as he can, turning his head slightly so he's staring right into his eyes. There's something in them, something that glints like little shards of broken glass that have sunk to the bottom of a bottle of vivid green poison, catching the light and shimmer



✨  
**Topic Probability**: 0.22622762803512286  
**Document**: It was an **inside** joke among the four Marauders that werewolves were well-endowed. They had no basis for comparison of course, having up until the age of twenty only ever known **one** werewolf, but from the first time Remus Lupin stepped from the shower stall in all his well-hung glory, the three boys agreed he was a beast in more ways than **one**.

Remus would blush brilliantly whenever conversation laced with innuendo about his wand length came up, but eventually he learned to take it all in stride and, quite like the jokes about his "furry little problem," teasing about his "impressive engorgement charm" became just as commonplace.

Of the four of them, however, Sirius joked about it the most. It didn't matter where they were, who was with them, or what they were doing--Sirius always found a way to work in some comment or innuendo.

When they would Apparate together: Didn't lose anything vital in that side-along did you? 

When they played pick-up Quidditch on the weekends: Foul! Moony is seeking, he isn't allowed to use a bat!

When they went out for drinks: They named the Three Broomsticks after our Remus here, you know. 

For his part, Remus would only laugh or smile and toss some silly comment back to deflect the attention away from him, but occasionally he'd tell Sirius to knock it off.

"What? Women always give you a second and third look whenever they hear us joke about it. You know you wouldn't stand a chance with me **around**. Gotta advertise, mate."

Remus rolls his eyes. "So you're just trying to give me **hand**?"

"It would take more than one **hand**, I'm sure"--Remus throws an empty takeaway carton at him as Sirius dissolves into giggles--"Of course I'm just trying to help. I'm very generous, Moony."

"Well that certainly explains why were drinking alone in your flat and not in the arms of some beautiful young thing. Anyway, I think you're overestimating how much women **want**...someone like me."

Sirius gives him an incredulous look. "All women **want** a huge **cock**, Remus. It's a fact."

"Considering you hardly need to learn a girl's name before you ask her back to your place, you'll forgive me if I find your credibility in how women think a bit lacking."

"Wanker," Sirius says, showing him two **fingers** and throwing the carton back.

Remus banishes it, along with the rest of the rubbish to the kitchen with a lazy flick of his wand and leans back into the squashy armchair, giving Sirius a patient look. "Women get a little intimidated when they see it actually. Like I'm going to hurt them or something. And no bloke will let me **fuck** him either."

At the end of their seventh year, Remus had confessed to his friends that he thought he preferred men. They all took it well for the most part. James insisted that as long as Remus still liked women, it was okay if he wanted to get off with a bloke every now and then, and Sirius logically reasoned that of course Remus would also like fucking men. There was plenty of him to go **around**, after all.

"It's more of a curse than a blessing," Remus says, wry smile twisting his **lips**, "believe me."

"You're having me on."

"Nope. I mean, I get by, you know," Remus assures, "but women...they think they want it, but when they're staring down the barrel they panic. And most guys get really insecure and insist I bottom. Which I don't really mind, but it would be nice to get off with someone without it being an issue, you know?"

"That is just..." Sirius shakes his head and throws back the remains of his firewhiskey. He reaches for the bottle on the floor and holds it out to Remus who shakes his head before pouring himself another. "I don't get it. That thing"--he gestures vaguely to Remus's crotch--"that thing should have its own religion. People should be on their knees worshipping your **cock** every day."

Remus laughs and takes a large swallow of his whiskey. "Really?"

Sirius nods in earnest. "Oh yes. Your cock is magnificent, Moony."

The second the w



✨  
**Topic Probability**: 0.21316823856749012  
**Document**: "Remember me?" 

Draco was not unaware of Potter's question, but he was busy. Busy staring at his **fingers**, three of them now, slick, with pink knuckles, disappearing **inside** Potter, the red puffy **skin** of Potter's anus wrapped **around** them, barely letting them move. Sliding them out was much easier; Potter's **body** pushed them out. It made the urge to push them back in even stronger.

The morning light provided a fine view. Every blush and freckle on Potter's **skin** was clearly visible. Potter was naked, lying on his stomach with a pillow pushed under his **hips**, his legs apart, wide enough for Draco to fit in between them.Draco was bent forward on his knees, with **one** **hand** on Potter's **arse** cheek, thumb pushing it wider apart, and the other... Well.

"Seriously. The rest of me still exists. Have you forgotten?"

The nonsensical statement had Draco looking up. "What?" The sound of his voice scared him; it came out raspy and broken. He would have cleared his throat and tried again, but the realisation that he was finger-fucking Potter struck him all over again and he was afraid to speak. Afraid he'd ruin it the way he used to ruin things and get into trouble the moment he opened his **mouth**. He could never trust his **mouth**; he'd learned that long ago.

Potter was looking at him over his shoulder, flushed, with strands of dark **hair** clinging to his face. His **lips** were curvedinto a teasing smile, which Draco could not comprehend. He was squinting without his glasses, bright green eyes narrowed to something darker. But there was something open and vulnerable in his face when he didn't have his glasses on. Or maybe it just seemed that way because Draco only saw Potter without his glasses when they were naked. It was no wonder Draco had developed a kink for it. Sometimes he'd see Potter on the other side of the Ministry's atrium and Potter would nod at him and fix his glasses, and it seemed like seduction, an invitation, shameless teasing. Baring Potter's **body**, baring his eyes, it all seemed so unattainable.

And yet here they were.

"Hi," Potter said. Sweetly, as though they had just met and he was trying to be polite. "I'm the bloke you've promised to **fuck**? A week or so ago, I think."

Draco nearly pointed out that, no, he promised no such thing a week ago. He promised it just this morning.

His mind cleared in time, though. "Conventionally, people say something like... Harder, faster, give it to me, yes, yes, now, **fuck** me with that big **cock**. Or prick. Or dick. What have you.Or they say, 'Hurry. Please.'"

Potter seemed to consider this for a moment. "Your **cock**. In me. Now," was what he came up with.

"Sounds like a telegram," Draco mused. "Lacks please."

"Would you like me to draw you a picture, too?"

Draco's **fingers** were still **inside** Potter. He pressed them down a little harder, twisted just a tiny bit.

Potter's eyelashes fluttered.

"I'm not sure how stick figures would help."

"Fine, then." Potter looked away, exhaled, rolled his **hips**. "This is doing it for me, anyway. Carry on."

That was Draco's plan, really. He'd make this last. He'd use his fingers to make Potter come. He'd get him **hard** again with his mouth. And then he'd fuck him. Then they'd have breakfast, and then he'd fuck him again.

But Draco had overestimated his patience. He pulled out, all the way this time.

Potter squirmed a little and made a breathy sound that had Draco's insides twisting. "God," he said, "you're so easy to manipulate."

A statement like that might have stung not that long ago. But back then Potter hadn't rolled onto his stomach and said, "Go on, then. Do it. I **want** you to," the way he had done this morning.

Draco placed his palms on Potter's **arse** cheeks, kneading them, spreading them apart. He probably shouldn't have been staring that much; it must have been strange, too telling. Potter was used to reverence; Draco wasn't supposed to be **one** of those p



✨  
**Topic Probability**: 0.20361146568598762  
**Document**: Draco was making the most delicious sounds. With each thrust of Harry's **cock** he gave a breathless, helpless little moan. Harry's eyes slipped shut as he moved, shielding him from the sight of Draco's **body** arching, pale **skin** flushed by lust and marked by Harry's **mouth**. But those moans Draco was making still assaulted him, along with the soft hot **skin** of Draco's **hard** **cock** in his **hand** and the slapping sounds as Harry pounded into him and Draco shoved up into every thrust. Harry's free **hand** tightened in the sheet as he tried to control himself. He couldn't pull himself out of this rhythm if he tried, Draco's bliss was ringing in his ears, but if he could just - hold - on -

Draco's moans became a guttural cry as he came. Harry gave himself up to it and they shuddered through orgasm together. Harry kissed Draco's **neck**, mouthing at the slick skin with its scruff, and coaxed another little shiver out of him.

They lay together, panting and sweaty and incapable of higher brain function. Draco tangled long **fingers** in Harry's **hair** and tugged him upwards until their mouths met. It was full of teeth clashing and slurping sounds, a long and sloppy and probably deeply unromantic **kiss**, except for being the kind that came after eleven months of mind-blowing sex.

Harry shifted a little and managed to pull out without giving up his lovely spot on top of Draco. He did pull back enough to break the **kiss**, and to watch Draco's blurred grey eyes open and blink up at him.

Harry loved Draco's sharp mind and fierce competitive instinct. Yet they only made it better when Harry could do this, get to see this: Draco dazed by pleasure, his sharp edges softened, the mind briefly overwhelmed by the **body**.

And the heart, Harry might have thought, except then Draco would sense it and mock him forever. As if Draco weren't incredibly soppy sometimes. He'd spent a full three hours introducing Harry to the Malfoy portraits and earnestly requesting their approval, hand clutching Harry's nervously tight. When Great-Uncle Septimus had said no, Draco had conjured a bucket of turpentine and waved it until his ancestor got with the programme.

Harry smiled besottedly down at Draco and shoved two **fingers** into his **arse**.

Draco's eyes widened, a small sound escaping him. He was hot **inside**, and slick with lube and Harry's come. Harry just kept his fingers there for a few moments, enjoying being **inside** Draco once more. Even better was holding Draco's eyes while he did it. Draco's body tautened a little from its post-orgasmic lassitude, his breathing speeding a little. Harry grinned, exulting in the slightly nervous anticipation in Draco's face as Harry flexed his fingers.

Harry watched every twitch and change in Draco's face as he fingered him slowly. He held Draco's gaze, refusing to let him hide the prickling, oversensitive pleasure Harry was inflicting. Draco turned his face away, closing his eyes. Harry pumped his fingers harder in retaliation, and Draco tightened round Harry's fingers.

"Merlin, Harry, I can't... I just came, I can't deal with..."

"All right," Harry told him. "Green, yellow, red?"

It was a sincere question, but there was an undeniable thrill in watching Draco's cheeks pinken as he was forced to admit he wanted this. "Green," he muttered.

"Pardon?" said Harry, scissoring his fingers.

"Green," Draco snarled, glaring. The glare collapsed into a groan as Harry slipped a third finger into him.

"See, there you are," Harry cooed. "I know you just can't get enough of this, of me fucking you til you can't see straight."

Draco laughed a little; the sound hitched as Harry nudged his prostate. "I never see straight, haven't you heard? When a hot man walks by I see anything but straight."

Harry smiled. "But it's just me who gets to drive you out of your fucking mind." He thrummed his fingers inside Draco. Draco's kiss-bruised **mouth** opened on a gasp.

"Celebrities," Draco huffed



✨  
**Topic Probability**: 0.20185973841502228  
**Document**: There were few things in the world that could get under the **skin** of **one** Harry Potter, Alpha Lord of the Ancient and Most Noble House of Black, Order of Merlin: First Class, and Savior of the Wizarding world. After facing life and death situations, toppling corrupt governments, and destroying dictators, there weren't many things that could make Lord Potter **want** to throw his flute of champagne across the floor and fly into a homicidal rage in the middle of the Ministry of Magic Solstice Ball.

And **one** of those such things happened to be situated on the chaise lounge in one of the ballroom alcoves.

Several young alphas were crowded **around** the chaise, practically reeking of pre-rut pheromones in the presence of a beautiful omega. And normally, Harry couldn't give a damn about a bunch of pups scrambling for any omega they could get their **hands** on.

Except it wasn't just any omega.

In the center of the alpha mob was Draco Malfoy, Omega jewel of the Black family.

In short, he was Harry's responsibility, as well as his intended.

He's doing this on purpose... Harry sighed, taking a sip of his champagne as he descended the staircase. Draco loved attention, and coming into his omega status had only made his drama queen nature worse. 

Sure enough, Draco was doing all he could to keep the young alpha's enthralled: batting his eyes, a coy nibble of his **lips**, brushing his **hand** across his pulse in the soft flesh of his **neck**. Each move was carefully planned and the alphas lapped it up like sweet wine.

Then Draco's eyes met Harry, and a smirk tugged at his **lips**.

Cheeky brat... Harry school his expression from showing annoyance, though he did raise his free **hand**, gesturing for Draco to come to him.

Draco only responded by smiling coyly, and giving a minuscule shake of his head.

Always wanting to do things the **hard** way... Harry sighed, passing his glass to a server as he walked by. With even, deliberate steps, Harry made his way to where his defiant ward was seated.

"Evening, gentlemen." Harry greeted the alphas with all the grace he could manage. "So nice of you to keep my Draco company."

Upon seeing the Conqueror of the Dark Lord, the alphas were quick to back off of their attentions to Draco. Harry couldn't help but smirk, placing a hand on the back of Draco's **neck**, "I hope I'm not interrupting, but I'm afraid I need to speak to Draco in private."

Harry didn't wait for them to respond, he had practically helped Draco to his feet and walked him away from the crowds.

"You're no fun, Harry..." Draco chuckled,swaying his **hips** slightly as they walked.

Harry didn't respond, simply turning to guide Draco away from the main ballroom and down one of the many hallways of the Ministry.

Draco was still giving Harry grief when he led both of them into an unoccupied washroom. Harry used the moment in which he locked the door and cast a silencing spell to take a calming breath.

After that, he struck.

Draco was cut mid-sentence as Harry bodily forced him against the wall, the blond's chest hitting the wall roughly.

"Harry!" Draco gasped once he was able to get his breath back, "What the hell are you doing?"

"You know exactly what I'm doing, you little tart."

There was a low growl that emanated from the alpha, causing Draco to tremble. Whether it was from fear or from excitement remained to be seen.

Harry grabbed both of Draco's slender wrists, pinning them to the wall above the blond's head easily with one hand. As Harry's free hand began to stroke down  Draco's stomach, "You just can't help yourself, can you Draco?"

A bright red blush crept across Draco's cheeks, "Harry..."

"What's this, acting coy all of a sudden?" Harry smirked, allowing his hand to wander along the insides of Draco's thighs.

The only response from Draco was a shaky gasp.

"What were you going to do with all those alphas, hm?" Harry murmured in Draco's ear, palming Draco's clothed crotch, "Were you going to mak

